In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [ ]:
TD = pd.read_excel('Tableau_Declarations-Prealables-Embauche.xlsx',header=0)
TD.head()

In [ ]:
TD.index

In [ ]:
mask_CDD = TD['Nature_de_contrat'].str.match("CDD.*")
CDD=TD[mask_CDD]
 
rng = pd.date_range('2000 Jan 1', periods=len(CDD), freq='MS')
CDD.index = rng
CDD


In [ ]:
mask_CDI = TD['Nature_de_contrat'].str.match("CDI.*")
CDI=TD[mask_CDI]
rng = pd.date_range('2000 Jan 1', periods=len(CDI), freq='MS')
CDI.index = rng
CDI.describe()

In [ ]:
ax= CDI.plot(y=["DPAE_brut"],style='-')
CDI.resample('BA').mean().plot(y=["DPAE_brut"],style=':',ax=ax,figsize=(20,8))

CDI.boxplot(column=["DPAE_brut"], by='Annee', figsize=(20,8))
CDI.boxplot(column=["DPAE_brut"], by='Trimestre', figsize=(10,8))


In [ ]:
ax= CDD.plot(y=["DPAE_brut"],style='-')
CDD.resample('BA').mean().plot(y=["DPAE_brut"],style=':',ax=ax,figsize=(20,8))

CDD.boxplot(column=["DPAE_brut"], by='Annee', figsize=(20,8))
CDD.boxplot(column=["DPAE_brut"], by='Trimestre', figsize=(10,8))


In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(CDD['DPAE_brut'][:70])
plt.show()
autocorrelation_plot(CDI['DPAE_brut'][:70])

In [ ]:
EES = pd.read_excel('Tableau_Emploi-Salarie-Trimestre-National_09032022.xlsx',index_col=0)

In [ ]:
EES.head(5)


In [ ]:
EES_A = EES.replace("(O)",pd.NA)

In [ ]:
EES_A.head()

In [ ]:
EES_A.isna().sum()
EES_NET = EES_A.drop(["idBank","Dernière mise à jour", "Période"],axis=1)

In [ ]:
EES_NET.head(5)

In [ ]:
EES_NET=EES_NET.dropna(axis=0)

In [ ]:
EES_NET.head(5)

In [ ]:
EES_NET.loc['total'] = EES_NET.sum(numeric_only=False, axis=0)

In [ ]:
EES_NET.head(5)

In [ ]:
EES_TOTAL = EES_NET.drop( EES_NET.index.to_list()[:-1] , axis = 0 )
EES_TOTAL

In [ ]:
EES_NET.index

In [ ]:
EES_NET[:4]

In [ ]:
secteur1 = EES_NET.iloc[[1]]
secteur2 = EES_NET.iloc[[2]]
secteur3 = EES_NET.iloc[[3]]
secteur4 = EES_NET.iloc[[4]]


In [ ]:
secteur1


In [ ]:
secteur2


In [ ]:
secteur3


In [ ]:
secteur4

Question 2 :

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(CDD["DPAE_brut"], period=12) 


plt.figure(figsize=(25,8))
plt.plot(CDD.index, CDD["DPAE_brut"], 'k')
plt.legend(['Cases'])
plt.figure(figsize=(25,8))
plt.plot(CDD.index,decomposition.trend)
plt.plot(CDD.index,decomposition.seasonal)
plt.plot(CDD.index,decomposition.resid)
plt.legend(['Trend', 'Seasonal', 'Resid'])

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(CDI["DPAE_brut"], period=12) 


plt.figure(figsize=(25,8))
plt.plot(CDI.index, CDI["DPAE_brut"], 'k')
plt.legend(['Cases'])
plt.figure(figsize=(25,8))
plt.plot(CDI.index,decomposition.trend)
plt.plot(CDI.index,decomposition.seasonal)
plt.plot(CDI.index,decomposition.resid)
plt.legend(['Trend', 'Seasonal', 'Resid'])